# Objetivo: Note base que implementa funciones para script procesador de productos
# Autor: Gustavo V. Diaz
# Fecha: 22/09/2024

### Descripción
Script base que servirá para copiar y pegar esctructura general y funciones desarrolladas en módulo especializado para las tareas necesarias.

Se parte de la hipótesis de tener definido un dataframe que contenga la búsqueda realizada por "searcher_filt.py" junto con el kml de búsqueda inicial. Posteriormente se irá recorriendo el dataframe para:
1. Bajar producto
2. Recortarlo
3. Ingresar polígono de búsqueda a producto SNAP
4. Aplicar resize a producto SNAP
5. Enmascarar cirros
6. Enmascarar por ROI de búsqueda
7. Procesar por NDVI o RGB
8. Entregar salida según se requiera en el paso 7

#### Entrada
1. df de productos a bajar (en csv o pkl)
2. Ruta kml de búsqueda

#### Salida

1. Carpeta de salida
    1. Composiciones RGB
        1. Tiff
        1. png
    2. Tabla metadatos
        1. Productos
            1. Id
            2. acq_date
            3. Tiff_Prod_name (salida)
            4. Ruta a Tiff_Prod_name
        2. Configuración
            1. Configuración de búsqueda
            2. Cantidad de productos procesados
            3. Fecha inicio análisis
            4. Fecha fin análisis
            5. Fecha procesamiento
            6. Nombre ROI